In [12]:
import os
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from scipy.stats import gaussian_kde
import seaborn as sns
from joblib import Parallel, delayed
from IPython.display import clear_output

import warnings

warnings.filterwarnings('ignore')
import pingouin as pg
from scipy import stats


1. RateConst Reader


In [13]:
def getAllRateInfo(file):   
    res=pd.DataFrame()
    try:
        temp=pd.read_csv(file)
        temp=temp.dropna()
        temp['Protein']=file.split('\\')[-1].replace('.RateConst.csv','')
        res=pd.concat([res,temp])
    except Exception as exp:
        return res
    
    return res
def getAllRate(data_path):
    results = Parallel(n_jobs=-1)(delayed(getAllRateInfo)(file) for file in [os.path.join(data_path,x) for x in os.listdir(data_path) if '.RateConst.csv' in x  ]) 
    all_data=pd.concat(results)
    all_data=all_data.reset_index(drop=True)
    all_data.columns=[x.strip() for x in all_data.columns]
    return all_data

2. Quant File Reader

In [14]:
def read_n_merge(skip,file):
    file_data=pd.read_csv(file,skiprows=skip,index_col=False)
    file_data.columns=[x.strip() for x in file_data.columns]
    isparsed=(set(['Peptide', 'UniqueToProtein', 'Exchangeable Hydrogens', 'Charge',
       'm/z(Sequence)', 'M0', 'M1', 'M2', 'M3', 'M4']).issubset(set(file_data.columns)))
    isparsed=isparsed and file_data.shape[0]>0    
    if isparsed:
        file_data['Protein']=file.split('\\')[-1].replace('.Quant.csv','') 
        return [isparsed,file_data]
    else:
        return [isparsed,None]
def get_df_all_quant_files(file):
    res=read_n_merge(1,file)
    all_data=None
    if res[0]: all_data=res[1]
    else: all_data=read_n_merge(3,file)[1]
    return all_data

def getquantfile(data_path):
    results = Parallel(n_jobs=-1)(delayed(get_df_all_quant_files)(file) for file in [os.path.join(data_path,x) for x in os.listdir(data_path) if '.Quant.csv' in x  ])
    all_data=pd.concat(results)
    all_data=all_data.reset_index(drop=True)
    all_data.columns=[x.strip() for x in all_data.columns]
    return all_data

3. Data source & path

In [ ]:
# organ="liver"
organ="heart"
# organ="muscle"
# data_path=r'C:\Workplace\Python\AnalysisForThePaper\NEH\d2ome_output\liverpool_liver'
data_path=r'C:\Workplace\Python\AnalysisForThePaper\NEH\d2ome_output\liverpool_heart'
# data_path=r'H:\Warehouse\Data\DataUsedForPublication\Partial IsotopeProfile paper data used for publication\liverpool_CI\muscle'


data_quant=getquantfile(data_path)
data_rate=getAllRate(data_path)
data_rate=data_rate[(data_rate.Rsquared!=' -nan(ind)')&(data_rate.Rsquared!=' ')]



4. merge quant and rate files

In [ ]:
merged=pd.merge(data_quant,
                data_rate,
                left_on=['Protein','Peptide','Charge'],
                right_on=['Protein','Peptides','Charge'])
merged=merged.reset_index(drop=True)
merged.shape

(13520, 217)

5. parameters

In [ ]:
pw,ph=0.046,1.5574E-4
# rsquared= 0.99
maxrate=math.log(2)
exp_time=[0 ,1 ,2 ,3 ,6 ,7 ,9 ,13,16,21,24,31]

6. filter data

In [ ]:
merged=merged[#(merged.Rsquared.astype('float')>=rsquared) &
              (merged.RateConstants< math.log(2))]
merged=merged.reset_index(drop=True)
merged.shape

(13240, 217)

7. Helper functions

In [ ]:
def get_I0_t(I0_0,I0_asymp,k,t):
    return I0_asymp + (I0_0-I0_asymp)*math.exp(-k*t)

def get_I0_asmyp(I0_0,neh):
    return I0_0*( (1 - pw/(1-ph))**neh )

def get_I0_exp(index,suffix):
    _sum=float(float(merged.loc[index,f'I0{suffix}'])+float(merged.loc[index,f'I1{suffix}'])+float(merged.loc[index,f'I2{suffix}'])+float(merged.loc[index,f'I3{suffix}'])+
                        float(merged.loc[index,f'I4{suffix}'])+float(merged.loc[index,f'I5{suffix}']))
    if _sum==0: return None
    else: return float(merged.loc[index,f'I0{suffix}'])/_sum

def getNewKestimate(M0_0,I0_0_exp,I0_asymp,t,I0_t_exp,numberofTerms): 
        
    base=(I0_0_exp-I0_t_exp)/(M0_0-I0_asymp)    
    new_kt= sum([ (base**i)/i for i in range(1,numberofTerms+1)])     
    new_k=new_kt/t    
    return new_k


def getNewKestimateByIndex(time,numberofterms_1,index,numberofterms_10):
        
        time_index=exp_time.index(time)
        suffix=f".{time_index}"  
        # print('time',time,time_index)
                                              
        if merged.loc[index,f'I0{suffix}'] == ' ' or merged.loc[index,f'I0'] == ' ': 
            return None            
        
        I0_0_exp= get_I0_exp(index,"")
        I0_t_exp=get_I0_exp(index,suffix)
        
        if I0_0_exp == None or I0_t_exp ==None or  I0_t_exp >  I0_0_exp: 
            return None

        M0_0=float(merged.loc[index,f'M0'])/100
        k=float(merged.loc[index,f'RateConstants'])
        I0_asymp=M0_0*( (1 - pw/(1-ph))**merged.loc[index,f'Exchangeable Hydrogens'] ) 
        i0_t_theo= I0_asymp + (M0_0-I0_asymp)*math.exp(-k*time)     
        
        exp_fsr= (I0_0_exp-I0_t_exp)/(M0_0-I0_asymp)
        # print(exp_fsr, (I0_0_exp-I0_t_exp),(M0_0-I0_asymp),"==>",(I0_0_exp,I0_t_exp),(M0_0,I0_asymp) )
        if exp_fsr > 1: return None
        exp_kt= -math.log( 1 - exp_fsr) 
        d2ome_kt=k*time
        
        error=   abs(exp_kt - d2ome_kt)/d2ome_kt
        
        

        return[merged.loc[index,'Protein'],
               merged.loc[index,'Peptide'],
               merged.loc[index,'Charge'],                
                merged.loc[index,f'Exchangeable Hydrogens'],
                M0_0,I0_0_exp,
                I0_asymp,
                I0_t_exp,i0_t_theo,k,exp_kt/time] +[getNewKestimate(M0_0,I0_0_exp,I0_asymp,time,I0_t_exp,numberofterms_1),
                                                    getNewKestimate(M0_0,I0_0_exp,I0_asymp,time,I0_t_exp,numberofterms_10),error]
        
def computeAllNewRates(_time,_numberOfTerms,_numberOfTerms10):
    # res = Parallel(n_jobs=-1)(delayed(getNewKestimateByIndex)(_time,_numberOfTerms,index) for index in range(merged.shape[0]))
    res=[getNewKestimateByIndex(_time,_numberOfTerms,index,_numberOfTerms10) for index in range(merged.shape[0])]
    res=pd.DataFrame([r for r in res if r!=None])
    res.columns=['Protein','Peptide','Charge',"NEH","M0","I0_0_exp","I0_asmp",'i0_t_exp','i0_t_theo','d2ome_k','k_sol','new_k_1','new_k_10','error']
    return res
  
def getStat(x,y):
    
    r=np.corrcoef(x,y)
    rd_k= (x - y)/x
    # # label=f"r={int(1000*r[0][1])/1000},μ = {int(100*np.mean(rd_k))/100}, median = {int(100*np.median(rd_k))/100}, sd = {int(100*np.std(rd_k))/100}"
    # label=[int(1000*r[0][1])/1000 , int(100*np.mean(rd_k))/100,int(100*np.median(rd_k))/100,int(100*np.std(rd_k))/100]    
        
    result=pg.corr(x, y, method='pearson')[['CI95%']].reset_index(drop=True)
    ci=result.loc[0,:][0]
    
    print(f"{r[0][1]:.2f} [{ci[0]:.2f},{ci[1]:.2f}]")
    
    label=[f"{r[0][1]:.2f} [{ci[0]:.2f},{ci[1]:.2f}]" , 
           int(100*np.mean(rd_k))/100,int(100*np.median(rd_k))/100,int(100*np.std(rd_k))/100]
    
        
    return label

8. Prepare all new k values

In [ ]:
allmerged=merged.copy()
allmerged.shape

(13240, 217)

In [ ]:
all=[]
for _time in tqdm([1 ,2 ,3 ,6 ,7,13,16]):
# for _time in tqdm([6 ,7,16]):    
    for _rsquared in [0.9][::-1]:
                        
            try:                
                merged=allmerged.copy()
                merged=merged[(merged.Rsquared.astype('float')>=_rsquared)]
                merged=merged.reset_index(drop=True)


                res=computeAllNewRates(_time,1,10)

                res=res[(res.d2ome_k.astype('float')>=0)]
                res=res[(res.new_k_1.astype('float')>=0)]
                res=res[(res.new_k_10.astype('float')>=0)]
                res=res[(res.k_sol.astype('float')>=0)]

                res=res[abs(res.M0 -res.I0_0_exp)/res.M0 <=0.1]
                
                val=[_time,res.shape[0]]+ getStat(res.d2ome_k,res.k_sol)+ getStat(res.d2ome_k,res.new_k_1)
                all.append(val)
                print(f"_time")

 
            except Exception as exp:
                print(exp)
                continue


 14%|█▍        | 1/7 [00:26<02:39, 26.65s/it]

name 'pg' is not defined


 29%|██▊       | 2/7 [00:45<01:50, 22.17s/it]

name 'pg' is not defined


 43%|████▎     | 3/7 [01:04<01:21, 20.44s/it]

...

name 'pg' is not defined


 86%|████████▌ | 6/7 [01:59<00:19, 19.18s/it]

name 'pg' is not defined


100%|██████████| 7/7 [02:17<00:00, 19.67s/it]

name 'pg' is not defined


In [ ]:
all=pd.DataFrame(all)
all.columns=['Labeling duration','Number of Peptides','Pearson correlation coefficient','Relative difference mean','Relative difference median','Relative difference SD',
             'Pearson correlation coefficient','Relative difference mean','Relative difference median','Relative difference SD']
all.to_csv(f"{organ}_stat.csv",index=False)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 10 elements